In [13]:
import re
from pathlib import Path

import pandas as pd

In [14]:
RESULTS_PATH = Path("misc")
list(RESULTS_PATH.glob("*"))

[PosixPath('misc/results_split_train.csv'),
 PosixPath('misc/results_floodnet_initial_msl.csv'),
 PosixPath('misc/results_floodnet_finetune.csv'),
 PosixPath('misc/results_single_double.csv'),
 PosixPath('misc/results_split_msl.csv'),
 PosixPath('misc/results_split_test.csv')]

In [15]:
(results_split_test := pd.read_csv(RESULTS_PATH / "results_split_test.csv").sort_index()).head(5)

,Name,State,Created,Runtime,epoch,challenge_score_safe,f1_loc_safe,f1_class_safe,f1_safe,f1_0_safe,...,precision_1_safe,precision_2_safe,precision_3_safe,precision_4_safe,recall_safe,recall_0_safe,recall_1_safe,recall_2_safe,recall_3_safe,recall_4_safe
0,t_split_wind_test_tuscaloosa_tornado_dahitra,finished,2024-12-03T00:13:05.000Z,35,0,0.528478,0.690531,0.459027,0.556773,0.988338,...,0.897529,0.414293,0.248631,0.532401,0.533510,0.996772,0.501600,0.263080,0.378131,0.527964
1,t_split_wind_test_tuscaloosa_tornado_farseg,finished,2024-12-03T00:12:20.000Z,35,0,0.638247,0.781246,0.576961,0.641202,0.991233,...,0.893528,0.615544,0.655819,0.727428,0.565043,0.997387,0.680097,0.452414,0.278402,0.416916
2,t_split_wind_test_tuscaloosa_tornado_baseline,finished,2024-12-03T00:11:32.000Z,37,0,0.617582,0.745655,0.562693,0.626893,0.989914,...,0.873865,0.575903,0.448367,0.643475,0.570353,0.996465,0.620689,0.416162,0.331104,0.487343
3,t_split_wind_test_joplin_tornado_dahitra,finished,2024-12-03T00:11:02.000Z,21,0,0.676649,0.828428,0.611601,0.664501,0.980171,...,0.779947,0.439318,0.356244,0.647005,0.703061,0.976781,0.713573,0.524618,0.604349,0.695984
4,t_split_wind_test_joplin_tornado_farseg,finished,2024-12-03T00:10:31.000Z,22,0,0.745332,0.885599,0.685218,0.725886,0.987012,...,0.841120,0.534599,0.429016,0.782386,0.746489,0.985404,0.808298,0.674043,0.588170,0.676532


In [16]:
(results_split_train := pd.read_csv(RESULTS_PATH / "results_split_train.csv").sort_index()).head(5)
# incomplete run
results_split_train = results_split_train[results_split_train.Name != "split_wind_test_tuscaloosa_tornado_baseline-2024-11-07T19:15:48"]

In [28]:
# TODO we need to manuall overwrite those NaNs in `best_challenge_score_target` for correct results
(results_split_msl := pd.read_csv(RESULTS_PATH / "results_split_msl.csv")).head(5)

,Name,State,Created,Runtime,epoch,challenge_score_target,best_challenge_score_target,best_challenge_score_target_epoch,f1_target,f1_loc_target,...,f1_target_3,f1_target_4,challenge_score_source,f1_loc_source,f1_source_class,f1_source_0,f1_source_1,f1_source_2,f1_source_3,f1_source_4
0,t_xbd_msl_split_wind_test_tuscaloosa_tornado_b...,finished,2024-12-25T23:20:24.000Z,3183,4.0,0.619612,0.646942,3.0,0.610845,0.759389,...,0.313602,0.502173,0.760366,0.840755,0.725914,0.989454,0.798094,0.583538,0.625339,0.762538
1,t_xbd_msl_split_wind_test_joplin_tornado_dahitra,crashed,2024-12-25T22:56:47.000Z,1395,1.0,0.638100,0.638100,0.0,0.625181,0.823970,...,0.421091,0.673713,0.698946,0.811517,0.650701,0.989719,0.712000,0.508531,0.578788,0.634943
2,t_xbd_msl_split_wind_test_joplin_tornado_farseg,finished,2024-12-25T22:13:14.000Z,2596,4.0,0.750269,0.753666,3.0,0.742308,0.884362,...,0.517658,0.776848,0.757651,0.859671,0.713928,0.991967,0.811055,0.598217,0.643437,0.650765
3,t_xbd_msl_split_wind_test_joplin_tornado_baseline,finished,2024-12-25T21:15:42.000Z,3418,4.0,0.672613,0.689924,3.0,0.670749,0.831379,...,0.410845,0.721646,0.662119,0.817658,0.595460,0.989053,0.717986,0.464583,0.427471,0.666128
4,t_xbd_msl_split_wind_test_moore_tornado_dahitra,finished,2024-12-25T20:39:42.000Z,2134,4.0,0.569821,NaN,NaN,0.618429,0.830931,...,0.403980,0.734775,0.677023,0.790807,0.628258,0.990317,0.679221,0.519623,0.500738,0.643113


In [18]:
def attach_index(df, name_pattern):
    name = df["Name"]
    match = name.str.extract(name_pattern)
    match.columns = ["split", "event", "model"]
    idx = pd.MultiIndex.from_frame(match)
    return df.set_index(idx)

In [19]:
split_msl_pattern = r"t_xbd_msl_split_((?:wind|flood|wildfire))_test_(.*)_((?:dahitra|farseg|baseline))"

# remove duplicated runs (in case of splits re-ran with 5 epochs), assuming the latest run is the proper one
# TODO tuscaloosa_tornado has missing re-runs!
split_msl_df = attach_index(results_split_msl, split_msl_pattern).drop(["State", "Created", "Runtime"], axis=1).sort_index()
split_msl_df = split_msl_df[~split_msl_df.index.duplicated(keep='first')].add_prefix("msl_")
split_msl_df

msl_Name  \
split    event               model                                                         
flood    hurricane_harvey    baseline  t_xbd_msl_split_flood_test_hurricane_harvey_ba...   
                             dahitra   t_xbd_msl_split_flood_test_hurricane_harvey_da...   
                             farseg    t_xbd_msl_split_flood_test_hurricane_harvey_fa...   
         hurricane_michael   baseline  t_xbd_msl_split_flood_test_hurricane_michael_b...   
                             dahitra   t_xbd_msl_split_flood_test_hurricane_michael_d...   
                             farseg    t_xbd_msl_split_flood_test_hurricane_michael_f...   
         midwest_flooding    baseline  t_xbd_msl_split_flood_test_midwest_flooding_ba...   
                             dahitra   t_xbd_msl_split_flood_test_midwest_flooding_da...   
                             farseg    t_xbd_msl_split_flood_test_midwest_flooding_fa...   
         nepal_flooding      baseline  t_xbd_msl_split_flood_test_nepal_flooding_base...   
                             dahitra   t_xbd_msl_split_flood_test_nepal_flooding_dahitra   
                             farseg     t_xbd_msl_split_flood_test_nepal_flooding_farseg   
wildfire pinery_bushfire     baseline  t_xbd_msl_split_wildfire_test_pinery_bushfire_...   
                             dahitra   t_xbd_msl_split_wildfire_test_pinery_bushfire_...   
                             farseg    t_xbd_msl_split_wildfire_test_pinery_bushfire_...   
         portugal_wildfire   baseline  t_xbd_msl_split_wildfire_test_portugal_wildfir...   
                             dahitra   t_xbd_msl_split_wildfire_test_portugal_wildfir...   
                             farseg    t_xbd_msl_split_wildfire_test_portugal_wildfir...   
         santa_rosa_wildfire baseline  t_xbd_msl_split_wildfire_test_santa_rosa_wildf...   
                             dahitra   t_xbd_msl_split_wildfire_test_santa_rosa_wildf...   
                             farseg    t_xbd_msl_split_wildfire_test_santa_rosa_wildf...   
         woolsey_fire        baseline  t_xbd_msl_split_wildfire_test_woolsey_fire_bas...   
                             farseg    t_xbd_msl_split_wildfire_test_woolsey_fire_farseg   
wind     hurricane_matthew   baseline  t_xbd_msl_split_wind_test_hurricane_matthew_ba...   
                             dahitra   t_xbd_msl_split_wind_test_hurricane_matthew_da...   
                             farseg    t_xbd_msl_split_wind_test_hurricane_matthew_fa...   
         hurricane_michael   baseline  t_xbd_msl_split_wind_test_hurricane_michael_ba...   
                             dahitra   t_xbd_msl_split_wind_test_hurricane_michael_da...   
                             farseg    t_xbd_msl_split_wind_test_hurricane_michael_fa...   
         joplin_tornado      baseline  t_xbd_msl_split_wind_test_joplin_tornado_baseline   
                             dahitra    t_xbd_msl_split_wind_test_joplin_tornado_dahitra   
                             farseg      t_xbd_msl_split_wind_test_joplin_tornado_farseg   
         moore_tornado       baseline   t_xbd_msl_split_wind_test_moore_tornado_baseline   
                             dahitra     t_xbd_msl_split_wind_test_moore_tornado_dahitra   
                             farseg       t_xbd_msl_split_wind_test_moore_tornado_farseg   
         tuscaloosa_tornado  baseline  t_xbd_msl_split_wind_test_tuscaloosa_tornado_b...   
                             dahitra   t_xbd_msl_split_wind_test_tuscaloosa_tornado_d...   
                             farseg    t_xbd_msl_split_wind_test_tuscaloosa_tornado_f...   

                                       msl_epoch  msl_challenge_score_target  \
split    event               model                                             
flood    hurricane_harvey    baseline        0.0                    0.256779   
                             dahitra         0.0                    0.250502   
                             farseg          0.0                    0.257967   

In [20]:
split_test_pattern = r"t_split_((?:wind|flood|wildfire))_test_(.*)_((?:dahitra|farseg|baseline))"
split_test_df = attach_index(results_split_test, split_test_pattern).drop(["State", "Created", "Runtime", "epoch"], axis=1).add_prefix("test_")

split_train_pattern = r"split_((?:wind|flood|wildfire))_test_(.*)_((?:dahitra|farseg|baseline))"
split_train_df = attach_index(results_split_train, split_train_pattern).drop(["State", "Created", "Runtime", "epoch"], axis=1).add_prefix("train_")


In [21]:
cat = pd.concat([split_test_df, split_msl_df, split_train_df], axis=1, join="outer")
cat

test_Name  \
split    event               model                                                         
wind     tuscaloosa_tornado  dahitra        t_split_wind_test_tuscaloosa_tornado_dahitra   
                             farseg          t_split_wind_test_tuscaloosa_tornado_farseg   
                             baseline      t_split_wind_test_tuscaloosa_tornado_baseline   
         joplin_tornado      dahitra            t_split_wind_test_joplin_tornado_dahitra   
                             farseg              t_split_wind_test_joplin_tornado_farseg   
                             baseline          t_split_wind_test_joplin_tornado_baseline   
         hurricane_matthew   dahitra         t_split_wind_test_hurricane_matthew_dahitra   
                             farseg           t_split_wind_test_hurricane_matthew_farseg   
                             baseline       t_split_wind_test_hurricane_matthew_baseline   
         moore_tornado       dahitra             t_split_wind_test_moore_tornado_dahitra   
                             farseg               t_split_wind_test_moore_tornado_farseg   
                             baseline           t_split_wind_test_moore_tornado_baseline   
         hurricane_michael   dahitra         t_split_wind_test_hurricane_michael_dahitra   
                             farseg           t_split_wind_test_hurricane_michael_farseg   
                             baseline       t_split_wind_test_hurricane_michael_baseline   
flood    midwest_flooding    dahitra         t_split_flood_test_midwest_flooding_dahitra   
                             farseg           t_split_flood_test_midwest_flooding_farseg   
                             baseline       t_split_flood_test_midwest_flooding_baseline   
         hurricane_harvey    dahitra         t_split_flood_test_hurricane_harvey_dahitra   
                             farseg           t_split_flood_test_hurricane_harvey_farseg   
                             baseline       t_split_flood_test_hurricane_harvey_baseline   
         nepal_flooding      dahitra           t_split_flood_test_nepal_flooding_dahitra   
                             farseg             t_split_flood_test_nepal_flooding_farseg   
                             baseline         t_split_flood_test_nepal_flooding_baseline   
         hurricane_michael   dahitra        t_split_flood_test_hurricane_michael_dahitra   
                             farseg          t_split_flood_test_hurricane_michael_farseg   
                             baseline      t_split_flood_test_hurricane_michael_baseline   
wildfire pinery_bushfire     dahitra       t_split_wildfire_test_pinery_bushfire_dahitra   
                             farseg         t_split_wildfire_test_pinery_bushfire_farseg   
                             baseline     t_split_wildfire_test_pinery_bushfire_baseline   
         santa_rosa_wildfire dahitra   t_split_wildfire_test_santa_rosa_wildfire_dahitra   
                             farseg     t_split_wildfire_test_santa_rosa_wildfire_farseg   
                             baseline  t_split_wildfire_test_santa_rosa_wildfire_base...   
         woolsey_fire        farseg            t_split_wildfire_test_woolsey_fire_farseg   
                             baseline        t_split_wildfire_test_woolsey_fire_baseline   
         portugal_wildfire   dahitra     t_split_wildfire_test_portugal_wildfire_dahitra   
                             farseg       t_split_wildfire_test_portugal_wildfire_farseg   
                             baseline   t_split_wildfire_test_portugal_wildfire_baseline   
         woolsey_fire        dahitra                                                 NaN   

                                       test_challenge_score_safe  \
split    event               model                                 
wind     tuscaloosa_tornado  dahitra                    0.528478   
                             farseg                     0.638247   
                             basel

In [40]:
df = cat[["train_challenge_score_safe", "test_challenge_score_safe"]].sort_index()
df.insert(2, "test_change", ((cat["test_challenge_score_safe"] - cat["train_challenge_score_safe"]) * 100 / cat["train_challenge_score_safe"]).round(2).astype(str) + "%")
df["msl_challenge_score_overall"] = cat["msl_best_challenge_score_target"].combine_first(cat["msl_challenge_score_target"])
df.insert(4, "msl_change", ((df["msl_challenge_score_overall"] - cat["test_challenge_score_safe"]) * 100 / cat["test_challenge_score_safe"]).round(2).astype(str) + "%")
df["nan_in_best"] = cat["msl_best_challenge_score_target"].isna()
df["msl_epoch"] = cat["msl_epoch"]
df

train_challenge_score_safe  \
split    event               model                                  
flood    hurricane_harvey    baseline                    0.559386   
                             dahitra                     0.570781   
                             farseg                      0.599403   
         hurricane_michael   baseline                    0.534152   
                             dahitra                     0.540322   
                             farseg                      0.498405   
         midwest_flooding    baseline                    0.580727   
                             dahitra                     0.566172   
                             farseg                      0.601033   
         nepal_flooding      baseline                    0.614268   
                             dahitra                     0.590617   
                             farseg                      0.648010   
wildfire pinery_bushfire     baseline                    0.369464   
                             dahitra                     0.339706   
                             farseg                      0.448631   
         portugal_wildfire   baseline                    0.399404   
                             dahitra                     0.398419   
                             farseg                      0.441592   
         santa_rosa_wildfire baseline                    0.000000   
                             dahitra                     0.346546   
                             farseg                      0.257622   
         woolsey_fire        baseline                    0.317406   
                             dahitra                     0.346546   
                             farseg                      0.385150   
wind     hurricane_matthew   baseline                    0.619577   
                             dahitra                     0.684927   
                             farseg                      0.718308   
         hurricane_michael   baseline                    0.671513   
                             dahitra                     0.704487   
                             farseg                      0.735634   
         joplin_tornado      baseline                    0.618264   
                             dahitra                     0.635981   
                             farseg                      0.670874   
         moore_tornado       baseline                    0.661363   
                             dahitra                     0.657030   
                             farseg                      0.688394   
         tuscaloosa_tornado  baseline                    0.678841   
                             dahitra                     0.674351   
                             farseg                      0.712326   

                                       test_challenge_score_safe test_change  \
split    event               model                                             
flood    hurricane_harvey    baseline                   0.247543     -55.75%   
                             dahitra                    0.239457     -58.05%   
                             farseg                     0.259401     -56.72%   
         hurricane_michael   baseline                   0.228499     -57.22%   
                             dahitra                    0.218013     -59.65%   
                             farseg                     0.236438     -52.56%   
         midwest_flooding    baseline                   0.276103     -52.46%   
                             dahitra                    0.263798     -53.41%   
                             farseg                     0.304396     -49.35%   
         nepal_flooding      baseline                   0.121020      -80.3%   
                             dahitra                    0.164128     -72.21%   
                             farseg                     0.286977     -55.71%   
wildfire pinery_bushfire     baseline                   0.315346     -14.65%   
         